In [ ]:
import glob
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import imutils
from rasterio import features

In [ ]:

def sufix(x):
    labels = {'1. Microaneurysms': '_MA', '2. Haemorrhages': '_HE', '3. Hard Exudates': '_EX', '4. Soft Exudates': '_SE', '5. Optic Disc': '_OD'}
    return labels[x]

# label encoding
def label_encoding(x):
    labels ={'1. Microaneurysms': 1, '2. Haemorrhages': 2, '3. Hard Exudates': 3, '4. Soft Exudates': 4, '5. Optic Disc': 0}
    return labels[x]



def extract_yolo(path, img_name, path_masks):

    if not os.path.exists(path):
        os.makedirs(path)


    for labelFolder in os.listdir(path_masks):
        print(labelFolder)
        for i in range(len(img_name)):   
            print(path_masks + '/' + labelFolder + '/' + img_name[i] + sufix(labelFolder) + '.tif')
            mask_image = cv2.imread(path_masks + '/' + labelFolder + '/' + img_name[i] + sufix(labelFolder) + '.tif')
            if mask_image is not None:
                mask_image = cv2.cvtColor(mask_image, cv2.COLOR_BGR2GRAY)
                mask_image = cv2.resize(mask_image, (640, 640))
                mask_image = (mask_image > 0).astype('uint8')*255

                shapes = list(features.shapes(mask_image, mask=(mask_image > 0)))

                all_coords = []
                for shape in shapes:
                    coords = shape[0]['coordinates'][0]
                    all_coords.append(coords)

                mask_coord = []
                for j in range(len(all_coords)):
                    label_line = str(label_encoding(labelFolder)) + ' ' + ' '.join(
                    [f'{int(cord[0]) / mask_image.shape[0]} {int(cord[1]) / mask_image.shape[1]}' for cord in all_coords[j]])
                    mask_coord.append(label_line)


                # Save bounding boxes to a file in YOLOv8 format
                with open(path + '/' + img_name[i] + '.txt', 'a', newline='') as f:
                    for masks_list in (mask_coord):
                        f.write(masks_list +'\n' )

                # Remove duplicate
                with open(path + '/' + img_name[i] + '.txt', 'r') as input_file:
                    lines = input_file.readlines()
                    # Remove duplicates from the list of lines
                    unique_lines = sorted(list(set(lines)))
                    # Open the output file and write the unique lines to it
                    with open(path + '/' + img_name[i] + '.txt', 'w') as output_file:
                        output_file.writelines(unique_lines)
                        
    return 


In [ ]:
# Salvar imagens com as dimensões 640 x 640 
def save_resize_img(dir_path):
    for image_path in glob.glob(dir_path + "/*"):
        img = cv2.imread(image_path)
        img = cv2.resize(img, (640, 640))
        cv2.imwrite(image_path, img)
    return 


# Ler o nome de todas as imagens do diretório
def image_name(dir_path):
    img_name = []
    for image_path in glob.glob(dir_path + "/*"):
        file = image_path.split("\\")[-1][:-4]
        img_name.append(file)
    return img_name

#---------------------------------------------------------------
path_train = 'yoloTypo_OD/train/images/'
path_val = 'yoloTypo_OD/valid/images/'
path_test = 'yoloTypo_OD/test/images/'

# salva as imagens redimensionadas para 640x640
save_resize_img(path_train)
save_resize_img(path_val)
save_resize_img(path_test)

# salva o nome das imagens de cada pasta
img_name_train = image_name(path_train)
img_name_val = image_name(path_val)
img_name_test = image_name(path_test)


In [ ]:
# pastas onde estão localizadas as máscaras 
path_masks_train = 'UNet-SegRD/datasets/test/masks/train/'
path_masks_val = 'UNet-SegRD/datasets/test/masks/val/'
path_masks_test = 'UNet-SegRD/datasets/test/masks/test/'


# pastas que serão saídas das labels, devem está no mesmo diretório que a pasta imagens
path_output_train = 'yoloTypo_OD/train/labels/'
path_output_val = 'yoloTypo_OD/valid/labels/'
path_output_test = 'yoloTypo_OD/test/labels/'

yolo_train = extract_yolo(path_output_train, img_name_train, path_masks_train)
yolo_val = extract_yolo(path_output_val, img_name_val, path_masks_val)
yolo_test = extract_yolo(path_output_test, img_name_test, path_masks_test)

